In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
DIR = '/kaggle/input/texts-classification-iad-hse-intro-2020/'

train = pd.read_csv(os.path.join(DIR, 'train.csv'))
test = pd.read_csv(os.path.join(DIR, 'test.csv'))

In [5]:
train.head()

,title,description,Category_name,Category
0,"Эбу Renault / Nissan 1,5 DCI Delphi 2001-2008 год","Комплект ЭБУ (мозги, компьютер мотора, двигате...",Запчасти и аксессуары,10
1,Утюг утп 1000 ватт СССР 1987 год,Продам/\n Фото № 1-2 /\n /\nУтюг УТП 1000 ватт...,Бытовая техника,21
2,Возвму машину с выкупом,Возьму машину в аренду с последующим выкупом н...,Предложение услуг,114
3,Полусапожки,"полусапожки в отличном состоянии, один раз оде...","Одежда, обувь, аксессуары",27
4,Босоножки кожаные,"Кожаные(натур) босоножки Karlo Pasolini, 40 рр...","Одежда, обувь, аксессуары",27


In [6]:
test.head()

,title,description,itemid
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804
3,Запчасти для GLE,Запчасти GLE,1689811299
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240


In [7]:
train.shape

(4234042, 4)

In [8]:
train.isnull().any()

title            False
description       True
Category_name    False
Category         False
dtype: bool

In [9]:
test.isnull().any()

title          False
description     True
itemid         False
dtype: bool

In [10]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [11]:
train.isnull().any()

title            False
description      False
Category_name    False
Category         False
dtype: bool

In [12]:
test.isnull().any()

title          False
description    False
itemid         False
dtype: bool

In [13]:
train['title&description'] = train['title'].str[:] + ' ' + train['description'].str[:]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(train[['title&description']], train['Category'], random_state=13)

In [15]:
tf_idf = TfidfVectorizer(max_features=25000)

In [16]:
X_train_tf_idf = tf_idf.fit_transform(X_train['title&description'])

In [17]:
X_val_tf_idf = tf_idf.transform(X_val['title&description'])

In [18]:
lr = LogisticRegression(random_state=13, solver='sag', n_jobs=-1)
lr.fit(X_train_tf_idf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=13,
                   solver='sag', tol=0.0001, verbose=0, warm_start=False)

In [19]:
y_val_pred = lr.predict(X_val_tf_idf)
accuracy_score(y_val_pred, y_val)

0.89897129080378

In [20]:
del X_train, y_train, X_val, y_val, X_train_tf_idf, X_val_tf_idf

In [21]:
test['title&description'] = test['title'].str[:] + ' ' + test['description'].str[:]

In [22]:
X_test_tf_idf = tf_idf.transform(test['title&description'])
y_test_pred = lr.predict(X_test_tf_idf)

In [23]:
ans = pd.DataFrame(columns=['Id', 'Category'])
ans['Id'] = test['itemid']

In [24]:
ans['Category'] = y_test_pred

In [25]:
ans.head()

,Id,Category
0,1778449823,27
1,1677656962,27
2,1758182804,32
3,1689811299,10
4,1804706240,30


In [26]:
ans.to_csv('submission_sag_nopr_25.csv', index=False)